In [22]:
import numpy as np
import matplotlib.pyplot as plt
import csky as cy

timer = cy.timing.Timer()
time = timer.time

# Trying to make our own dataset

Our data:
- We are using QUESO Upgrade MC data /data/sim/IceCubeUpgrade/genie/level4_queso/ (120028, 121028, 140028, 141028, 160028, 161028) & /data/sim/IceCubeUpgrade/muongun/level4_queso/130028
- These are converted into .npy files via this converter: https://github.com/mjlarson/nusources_dataset_converters/blob/main/convert_upgrade_queso.py

Our references for looking up csky related stuff:
- We used csky's tutorials GettingStarted (https://docs.icecube.aq/csky/main/01_getting_started.html) and Implementation Inspection (https://docs.icecube.aq/csky/main/06_implementation_inspection.html) as a guideline
- We also looked at other analyses (e.g. https://github.com/icecube/wg-nu-sources/blob/main/2022_GRECO_GRB/scripts/01_get_no_prior_background_allsky_scan.py)


## Change keys and reformat input data

In [58]:
# Reading in the signal and background files and change the keys to the correct ones --> so that csky can read it

def get_signal_files(key):
    dir = "/data/user/bschlueter/software/nusources_dataset_converters/data/queso_"+str(key)+".npy"
    sig = np.load(dir)
    sig = np.hstack(sig)
    sig_upgraded = np.lib.recfunctions.rename_fields(sig, {'ow': 'oneweight','trueE':'true_energy'})
    sig_upgraded = sig_upgraded.view(np.recarray)
    return sig_upgraded

#Background
bg_filename = "/data/user/bschlueter/software/nusources_dataset_converters/data/muongun_131028.npy"
bg = np.load(bg_filename)
bg_upgraded = np.lib.recfunctions.rename_fields(bg, {'atmo_weight': 'weight'})
bg_upgraded = bg_upgraded.view(np.recarray)

In [48]:
live = 8*60*60
keys = ['120028','121028','140028','141028','160028','161028']
Dataset = []
for key in keys:  
    Dataset.append(cy.selections.Dataset(key = key,livetime=live, sig= get_signal_files(key), bg = bg_upgraded))

In [49]:
Dataset

[Dataset('120028', livetime = 28800s) with  and 0.0006169195555999903 expected bg events and sum(ow*E^-2) = 1.60,
 Dataset('121028', livetime = 28800s) with  and 0.0006169195555999903 expected bg events and sum(ow*E^-2) = 0.41,
 Dataset('140028', livetime = 28800s) with  and 0.0006169195555999903 expected bg events and sum(ow*E^-2) = 1.46,
 Dataset('141028', livetime = 28800s) with  and 0.0006169195555999903 expected bg events and sum(ow*E^-2) = 1.47,
 Dataset('160028', livetime = 28800s) with  and 0.0006169195555999903 expected bg events and sum(ow*E^-2) = 1.02,
 Dataset('161028', livetime = 28800s) with  and 0.0006169195555999903 expected bg events and sum(ow*E^-2) = 0.50]

Since we don't have data to input, we are missing 'with XXX data events and ...' in the output

In [50]:
with time('ana setup (from scratch)'):
    ana = cy.get_analysis(cy.selections.repo,spec = Dataset)

Setting up Analysis for:

Done.

0:00:00.000527 elapsed.


## Try to do anything with it

In [55]:
cy.CONF['ana'] = ana
cy.CONF['mp_cpus'] = 3

src = cy.sources(180, 0, deg=True)
tr = cy.get_trial_runner(src=src)

In [54]:
bg = cy.dists.Chi2TSD(tr.get_many_fits(1))

Performing 1 background trials using 1 core:
          0/1 trials complete...

IndexError: list index out of range

# Fake GRL in case it is needed

In [57]:
#fake GRL 
current_mjd = 60369
runstart = 135328
run_duration = 1/3 #8h
nr_runs = 2000
background_event_rate = 4.587 #calculated in Greco_vs_Queso.ipynb

new_grl_entries = []
dtype = [('run', int), ('start', np.float64), ('stop', np.float64), ('livetime', np.float64), ('events', int)]
for i in range(runstart, runstart+nr_runs+1):
    new_grl_entries.append((i, current_mjd, current_mjd + run_duration, run_duration, background_event_rate))
    current_mjd += run_duration

grl = np.array(new_grl_entries, dtype=dtype)
grl

array([(135328, 60369.        , 60369.33333333, 0.33333333, 4),
       (135329, 60369.33333333, 60369.66666667, 0.33333333, 4),
       (135330, 60369.66666667, 60370.        , 0.33333333, 4), ...,
       (137326, 61035.        , 61035.33333334, 0.33333333, 4),
       (137327, 61035.33333334, 61035.66666667, 0.33333333, 4),
       (137328, 61035.66666667, 61036.        , 0.33333333, 4)],
      dtype=[('run', '<i8'), ('start', '<f8'), ('stop', '<f8'), ('livetime', '<f8'), ('events', '<i8')])